# Chapter 7. 제너레이터 사용하기

## 제너레이터 만들기

+ 제너레이터는 한번에 하나씩 구성요소를 반환해주는 이터러블을 생성해주는 객체
+ 메모리를 적게 사용하는 반복을 위한 방법으로 2001년에 소개됨
+ <b>yield</b> 키워드를 사용하면 제너레이터 함수가 됨
+ <a href="https://medium.com/@silmari/python-tempfile-%EC%9E%84%EC%8B%9C%ED%8C%8C%EC%9D%BC-%EB%B0%8F-%ED%8F%B4%EB%8D%94-%EB%A7%8C%EB%93%A4%EA%B8%B0-86ea533086ce"> gettemdir 설명 </a>

In [1]:
import os
from tempfile import gettempdir

PURCHASES_FILE = os.path.join(gettempdir(), "purchases.csv")


def create_purchases_file(filename, entries=1_000_000):
    if os.path.exists(PURCHASES_FILE):
        return

    with open(filename, "w+") as f:
        for i in range(entries):
            line = f"2018-01-01,{i}\n"
            f.write(line)

In [2]:
import logging

logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)

class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()

    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )


def _load_purchases(filename):
    """
    list 사용
    """
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))

    return purchases


def load_purchases(filename):
    """
    제너레이터
    """
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)


def main():
    create_purchases_file(PURCHASES_FILE)
    purchases = load_purchases(PURCHASES_FILE)
    stats = PurchasesStats(purchases).process()
    logger.info("Results: %s", stats)


if __name__ == "__main__":
    main()

Results: PurchasesStats(0.0, 999999.0, 499999.5)


### 제너레이터 표현식
```
sum(x**2 for x in range(10)
```

## 이상적인 반복
### 관용적인 반복코드

In [3]:
class SequenceOfNumber():

    def __init__(self, start=0):
        self.current = start
        
    def __next__(self):
        current, self.current = self.current, self.current + 1
        return current
    
    def __iter__(self):
        return self
    
list(zip(SequenceOfNumber(0), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

#### next함수
+ 다음 요소로 이동시키고 기존의 값을 반환
+ 이터레이터가 더 이상의 값을 가지고 있지 않다면 StopIteration 예외가 발생함
+ StopIteration 예외 대신 기본값을 반환할 수 있음

In [4]:
word = iter("abc")
next(word)
next(word)
next(word)

'c'

In [5]:
next(word)

StopIteration: 

In [ ]:
next(word, "default value")

#### 제너레이터 사용하기
+ yield를 활용하여 클래스 사용없이 간단하게 구현 가능

In [ ]:
def sequnce(num):
    curr = 0
    while(True):
        retrun_val, curr = curr, curr + 1
        yield retrun_val
        
list(zip(sequnce(0), "abcdef"))        

#### itertools
+ islice : 원하는 갯수만큼 잘라서 반환
+ tee : 이터러블을 원하는 수만큼 복제

In [ ]:
""" 특정 조건의 가격만 계산하기 """
from itertools import islice, tee

purchases = islice(filter(lambda x: x > 1000.0, load_purchases(PURCHASES_FILE)), 10) # 처음 10개만 처리
stats = PurchasesStats(purchases).process()
logger.info("Results: %s", stats)


""" 이터러블 복제하기 """
def process_purchases(purchases):
    min_, max_ = tee(purchases, 2)
    return min(min_), max(max_)

purchases = load_purchases(PURCHASES_FILE)
logger.info("Results: %d, %d", *process_purchases(purchases))

#### 중첩 루프

In [6]:
def search_nested_bad(array, desired_value):
    """Example of an iteration in a nested loop."""
    coords = None
    for i, row in enumerate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break

        if coords is not None:
            break

    if coords is None:
        raise ValueError(f"{desired_value} not found")

    logger.info("value %r found at [%i, %i]", desired_value, *coords)
    return coords


def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell


def search_nested(array, desired_value):
    """"Searching in multiple dimensions with a single loop."""
    try:
        coord = next(
            coord
            for (coord, cell) in _iterate_array2d(array)
            if cell == desired_value
        )
    except StopIteration:
        raise ValueError(f"{desired_value} not found")

    logger.info("value %r found at [%i, %i]", desired_value, *coord)
    return coord


if __name__ == "__main__":
    array2d = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]
    # search_nested_bad(array2d, 13) 같은 결과이나 코드가 복잡
    search_nested(array2d, 12)

value 12 found at [2, 3]


### 파이썬의 이터레이터 패턴
+ \_\_iter\_\_ 매직 메서드를 통해 이터레이터를 반환하고, \_\_next\_\_ 매직 메서드를 통해 반복 로직을 구현
+ 모든 제너레이터는 이터레이터임
+ \_\_len()\_\_, \_\_getitem\_\_()를 구현한 시퀀스 객체도 반복 가능함

In [7]:
class SequenceWrapper:
    def __init__(self, original_sequence):
        self.seq = original_sequence

    def __getitem__(self, item):
        value = self.seq[item]
        logger.info("%s getting %s", self.__class__.__name__, item)
        return value

    def __len__(self):
        return len(self.seq)


class MappedRange:
    """Apply a transformation to a range of numbers."""

    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)

    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        logger.info("Index %d: %s", index, result)
        return result

    def __len__(self):
        return len(self._wrapped)

In [8]:
seq = SequenceWrapper((0,1,2,3,4,5,6,7,8,9))
mr = MappedRange(abs, 2, 5)
print(mr[0])
list(mr)

Index 0: 2
Index 0: 2
Index 1: 3
Index 2: 4


2


[2, 3, 4]

## 코루틴(coroutine)

+ 코루틴의 인터페이스
    + close()
        + 제너레이터에서 GeneratorExit 예외가 발생하며, 이 예외를 따로 처리하지 않으면 제너레이터 더 이상 값을 생성하지 않으며 반복이 중지됨
    + throw(ex_type[, ex_value[, ex_traceback]])
        + 현재 제너레이터가 중단된 위치에서 예외를 던짐
    + send(value)
        + 제너레이터와 코루틴을 구분하는 기준으로 yield 키워드가 할당 구문의 오른쪽에 나오게 되고 인자 값을 받아서 다른 곳에서 할당할 수 있음을 뜻함
        + next()를 먼저 호출해야 함
        + next()를 자동 호출해주는 @prepare_coroutine을 사용하면 편리함

In [9]:
""" Stop exception을 활용한 리턴 방법(실제로 쓰이진 않음) """
def generator():
    yield 1
    yield 2
    return 3

gen = generator()

while(True):
    try:
        next(gen)
    except StopIteration as e:
        print(">>> returned value", e.value)
        break

>>> returned value 3


In [10]:
""" 간단한 yield from 사례 """ 
def chain(*iterables):
    for it in iterables:
        for values in it:
            yield value

In [11]:
def chain(*iterables):
    for it in iterables:
        yield from it # 

In [12]:
list(chain("hello", ["world"], ("tuple", "value")))

['h', 'e', 'l', 'l', 'o', 'world', 'tuple', 'value']

In [13]:
""" 서브 제너레이터에서 반환된 값 구하기
yield from을 사용하면 코루틴의 종료 시 최종 반환 값을 구할 수 있음
"""

def sequence(name, start, end):
    logger.info("%s started at %i", name, start)
    yield from range(start, end)
    logger.info("%s finished at %i", name, end)
    return end

def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

g = main()

while(True): # yield from을 처음부터 쓸 수는 없음
    try:
        next(g)
    except StopIteration as e:
        print(">>> returned value", e.value) # 리턴값을 반환받을 수 있음
        break

first started at 0
first finished at 5
second started at 5
second finished at 10


>>> returned value 15


In [14]:
""" 서브 제너레이터와 테이터 송수신하기 """

class CustomException(Exception):
    """A type of exception that is under control."""


def sequence(name, start, end):
    value = start
    logger.info("%s started at %i", name, value)
    while value < end:
        try:
            received = yield value
            logger.info("%s received %r", name, received)
            value += 1
        except CustomException as e:
            logger.info("%s is handling %s", name, e)
            received = yield "OK"
    return end


def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)

if __name__ == "__main__":
    print(list(main()))

first started at 0
first received None
first received None
first received None
first received None
first received None
second started at 5
second received None
second received None
second received None
second received None
second received None


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
